In [10]:
import logging
import timm
import os
import random
import torch
import pandas as pd
import numpy as np
from albumentations.pytorch import ToTensorV2
import albumentations as A
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import platform

os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'
    
TRAIN_KR_IMAGE_PATH = PRE_PATH + 'data/train_kr'
TRAIN_KR_CSV_PATH = PRE_PATH + 'data/train_kr.csv'
MODEL_FILE = PRE_PATH + 'submission/0.9190/' + 'efficientnet_b4_Ep7_L_0.5274_A_0.9054_F1_0.9024.pt'
MODEL_SAVE_PATH = PRE_PATH + 'evaluation/'

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NUM_CLASSES = 17
BATCH_SIZE = 32
NUM_WORKERS = 0  # 일단 0으로 설정하여 단일 프로세스로 수행

In [11]:
def create_directory_with_backup(path):
    try:
        if os.path.exists(path):
            backup_path = path + '_backup'
            os.rename(path, backup_path)
            print(f"Existing folder renamed to: {backup_path}")
        os.makedirs(path)
        print(f"Folder created: {path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Dataset class 정의
class ImageDataset(Dataset):
    def __init__(self, df, image_path, transform=None):
        self.df = df
        self.image_path = image_path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        try:
            name, target = self.df.iloc[idx]
            img = np.array(Image.open(os.path.join(self.image_path, name)).convert("RGB"))

            # 변환 적용
            if self.transform:
                img = self.transform(image=img)['image']

            return img, target

        except Exception as e:
            print(f"Error loading image {self.df.iloc[idx, 0]}: {e}")
            return torch.zeros(3, 380, 380), -1

# 이미지 변환 설정
def get_transforms():
    valid_transform = A.Compose([
        A.Resize(380, 380),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    return valid_transform

# 특정 클래스에 대한 confidence 계산 및 출력
def calculate_class_confidence(model, loader, target_class):
    model.eval()
    results = []
    with torch.no_grad():
        tbar = tqdm(loader, desc="Calculating Confidence")
        for images, targets in tbar:
            images = images.to(device)
            outputs = model(images)
            confidences = torch.softmax(outputs, dim=1)
            target_confidences = confidences[:, target_class].detach().cpu().numpy()
            for idx, confidence in enumerate(target_confidences):
                results.append((loader.dataset.df.iloc[idx, 0], confidence))
    
    results = sorted(results, key=lambda x: x[1], reverse=True)
    for filename, confidence in results:
        if confidence < 0.8:
            print(f"Image: {filename}, Confidence: {confidence:.4f}")

# 메인 함수
def run_confidence_check(target_class):
    train_df = pd.read_csv(TRAIN_KR_CSV_PATH)

    # 클래스가 3인 이미지들만 필터링
    target_df = train_df[train_df['target'] == target_class]

    valid_transform = get_transforms()

    train_dataset = ImageDataset(target_df, TRAIN_KR_IMAGE_PATH, transform=valid_transform)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    model_name = 'tf_efficientnet_b4'
    model = timm.create_model(model_name, pretrained=False, num_classes=NUM_CLASSES, in_chans=3)

    # map_location 인자를 사용하여 모델을 CPU에서 로드
    model.load_state_dict(torch.load(MODEL_FILE, map_location=device))
    model = model.to(device)

    # 특정 클래스에 대한 confidence 계산 및 출력
    calculate_class_confidence(model, train_loader, target_class)

In [12]:
# target_class = 3  # 여기서 특정 클래스를 설정
for target_class in range(17):
    run_confidence_check(target_class)

Calculating Confidence: 100%|██████████| 4/4 [00:24<00:00,  6.06s/it]


Image: 108c622ecd7a9ea0_계좌번호.jpg, Confidence: 0.7737
Image: 07a0a849cab135e0_계좌번호.jpg, Confidence: 0.7231


Calculating Confidence: 100%|██████████| 4/4 [00:24<00:00,  6.02s/it]


Image: 25cb3d4a2e9d24e2_입퇴원 확인서.jpg, Confidence: 0.5398
Image: 51fe52a95d431a86_입퇴원 확인서.jpg, Confidence: 0.4879
Image: 337eb06c7c2d709d_입퇴원 확인서.jpg, Confidence: 0.2414
Image: 4620f6e53442f3b6_입퇴원 확인서.jpg, Confidence: 0.0158
Image: 0808c7453316041c_입퇴원 확인서.jpg, Confidence: 0.0000
Image: 45f0d2dfc7e47c03_입퇴원 확인서.jpg, Confidence: 0.0000
Image: 1c893f02de43b62a_입퇴원 확인서.jpg, Confidence: 0.0000


Calculating Confidence: 100%|██████████| 4/4 [00:22<00:00,  5.64s/it]


...

Calculating Confidence: 100%|██████████| 3/3 [00:16<00:00,  5.62s/it]


Image: 0e73f69b538fb276_이력서.jpg, Confidence: 0.7875
Image: 44a6366325306cc6_이력서.jpg, Confidence: 0.6060
Image: 139fec9a67910e28_이력서.jpg, Confidence: 0.5019
Image: 2b1076abe3e4338d_이력서.jpg, Confidence: 0.2954
Image: 363edeab084e0800_이력서.jpg, Confidence: 0.0000


Calculating Confidence: 100%|██████████| 2/2 [00:11<00:00,  5.53s/it]


Image: 8db8f37ec84b4f8f_소견서.jpg, Confidence: 0.2176
Image: 45ec22e6b1b60734_소견서.jpg, Confidence: 0.0474
Image: 3696b2ffaa6147e6_소견서.jpg, Confidence: 0.0377
Image: 840c53f776730ffe_소견서.jpg, Confidence: 0.0013
Image: 1ec14a14bbe633db_소견서.jpg, Confidence: 0.0000


Calculating Confidence: 100%|██████████| 4/4 [00:24<00:00,  6.04s/it]
